# Single neural network layer using Flux.jl

## Read in and process data

In [29]:
using Flux
using Flux: onehot

In [59]:
using CSV

apples_1 = CSV.read("Apple_Golden_1.dat", delim='\t')
apples_2 = CSV.read("Apple_Golden_2.dat", delim='\t')
apples_3 = CSV.read("Apple_Golden_3.dat", delim='\t')
bananas = CSV.read("Banana.dat", delim='\t')
grapes_1 = CSV.read("Grape_White.dat", delim='\t')
grapes_2 = CSV.read("Grape_White_2.dat", delim='\t');

apples = vcat(apples_1, apples_2, apples_3)
grapes = vcat(grapes_1, grapes_2);

In [3]:
col1 = :red
col2 = :blue

x_apples  = [ [apples_1[i, col1], apples_1[i, col2]] for i in 1:size(apples_1)[1] ]
append!(x_apples, [ [apples_2[i, col1], apples_2[i, col2]] for i in 1:size(apples_2)[1] ])
append!(x_apples, [ [apples_3[i, col1], apples_3[i, col2]] for i in 1:size(apples_3)[1] ])

x_bananas = [ [bananas[i, col1], bananas[i, col2]] for i in 1:size(bananas)[1] ]

x_grapes = [ [grapes_1[i, col1], grapes_1[i, col2]] for i in 1:size(grapes_1)[1] ]
append!(x_grapes, [ [grapes_2[i, col1], grapes_2[i, col2]] for i in 1:size(grapes_2)[1] ])

xs = vcat(x_apples, x_bananas, x_grapes);

We now we wish to classify the three types of fruit, so we will use as output *one-hot vectors*. Effectively, the first neuron learns whether (1) or not (0) the data corresponds to an apple, the second whether (1) or not (0) it corresponds to a banana, etc:

In [35]:
labels = [ones(length(x_apples)); 2*ones(length(x_bananas)); 3*ones(length(x_grapes))];

ys = [onehot(label, 1:3) for label in labels]  # onehotbatch(labels, 1:3)

2935-element Array{Flux.OneHotVector,1}:
 Bool[true, false, false]
 Bool[true, false, false]
 Bool[true, false, false]
 Bool[true, false, false]
 Bool[true, false, false]
 Bool[true, false, false]
 Bool[true, false, false]
 Bool[true, false, false]
 Bool[true, false, false]
 Bool[true, false, false]
 Bool[true, false, false]
 Bool[true, false, false]
 Bool[true, false, false]
 ⋮                       
 Bool[false, false, true]
 Bool[false, false, true]
 Bool[false, false, true]
 Bool[false, false, true]
 Bool[false, false, true]
 Bool[false, false, true]
 Bool[false, false, true]
 Bool[false, false, true]
 Bool[false, false, true]
 Bool[false, false, true]
 Bool[false, false, true]
 Bool[false, false, true]

In [32]:
onehot(1, 1:3)

3-element Flux.OneHotVector:
  true
 false
 false

The input data is in `xs` and the one-hot vectors are in `ys`.

## Single layer in Flux

Let's suppose that there are two pieces of input data. Then the network has 2 input neurons and 3 output neurons:

In [9]:
include("draw_neural_net.jl")

draw_layer (generic function with 1 method)

In [11]:
plot()
draw_layer(1, 1, 2, 3, 0.2)
plot!()

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polygon clip-path="url(#clip5202)" points="
264.386,269.683 264.235,266.654 263.781,263.656 263.031,260.717 261.991,257.868 260.672,255.138 259.087,252.552 257.252,250.138 255.185,247.919 252.906,245.917 
 250.439,244.153 247.809,242.644 245.041,241.406 242.163,240.449 239.204,239.785 236.193,239.42 233.161,239.357 230.138,239.597 227.154,240.137 224.238,240.973 
 221.421,242.096 218.73,243.494 216.192,245.154 213.832,247.059 211.675,249.19 209.741,251.526 208.049,254.043 206.618,256.717 205.46,259.52 204.589,262.424 
 204.011,265.402 203.734,268.422 203.759,271.454 204.087,274.469 204.715,277.436 205.635,280.326 206.84,283.109 208.316,285.758 210.049,288.246 212.022,290.549 
 214.216,292.644 216.607,294.509 219.173,296.126 221.887,297.479 224.723,298.554 227.651,299.341 230.644,299.831 233.671,300.02 236.702,299.906 239.706,299.49 
 242.653,298.776 245.514,297.772 248.261,296.487 250.866,294.934 253.303,293.128 255.547,291.089 257.577,288.835 259.372,286.391 260.913,283.779 262.186,281.026 
 263.178,278.16 263.879,275.21 264.281,272.204 264.386,269.683 
 " fill="#0000ff" fill-opacity="0.5"/>
<polyline clip-path="url(#clip5202)" style="stroke:#000000; stroke-width:1; stroke-opacity:0.5; fill:none" points="
 264.386,269.683 264.235,266.654 263.781,263.656 263.031,260.717 261.991,257.868 260.672,255.138 259.087,252.552 257.252,250.138 255.185,247.919 252.906,245.917 
 250.439,244.153 247.809,242.644 245.041,241.406 242.163,240.449 239.204,239.785 236.193,239.42 233.161,239.357 230.138,239.597 227.154,240.137 224.238,240.973 
 221.421,242.096 218.73,243.494 216.192,245.154 213.832,247.059 211.675,249.19 209.741,251.526 208.049,254.043 206.618,256.717 205.46,259.52 204.589,262.424 
 204.011,265.402 203.734,268.422 203.759,271.454 204.087,274.469 204.715,277.436 205.635,280.326 206.84,283.109 208.316,285.758 210.049,288.246 212.022,290.549 
 214.216,292.644 216.607,294.509 219.173,296.126 221.887,297.479 224.723,298.554 227.651,299.341 230.644,299.831 233.671,300.02 236.702,299.906 239.706,299.49 
 242.653,298.776 245.514,297.772 248.261,296.487 250.866,294.934 253.303,293.128 255.547,291.089 257.577,288.835 259.372,286.391 260.913,283.779 262.186,281.026 
 263.178,278.16 263.879,275.21 264.281,272.204 264.386,269.683 
 "/>
<polygon clip-path="url(#clip5202)" points="
264.386,117.986 264.235,114.957 263.781,111.959 263.031,109.02 261.991,106.171 260.672,103.441 259.087,100.855 257.252,98.4408 255.185,96.2219 252.906,94.2203 
 250.439,92.4563 247.809,90.9473 245.041,89.7085 242.163,88.7522 239.204,88.0881 236.193,87.7226 233.161,87.6595 230.138,87.8995 227.154,88.4401 224.238,89.2758 
 221.421,90.3985 218.73,91.7968 216.192,93.4567 213.832,95.3618 211.675,97.4929 209.741,99.8287 208.049,102.346 206.618,105.02 205.46,107.823 204.589,110.727 
 204.011,113.705 203.734,116.724 203.759,119.757 204.087,122.772 204.715,125.739 205.635,128.629 206.84,131.412 208.316,134.061 210.049,136.549 212.022,138.852 
 214.216,140.947 216.607,142.812 219.173,144.429 221.887,145.782 224.723,146.857 227.651,147.644 230.644,148.134 233.671,148.323 236.702,148.209 239.706,147.793 
 242.653,147.079 245.514,146.075 248.261,144.79 250.866,143.237 253.303,141.431 255.547,139.392 257.577,137.138 259.372,134.693 260.913,132.082 262.186,129.329 
 263.178,126.463 263.879,123.513 264.281,120.507 264.386,117.986 
 " fill="#0000ff" fill-opacity="0.5"/>
<polyline clip-path="url(#clip5202)" style="stroke:#000000; stroke-width:1; stroke-opacity:0.5; fill:none" points="
 264.386,117.986 264.235,114.957 263.781,111.959 263.031,109.02 261.991,106.171 260.672,103.441 259.087,100.855 257.252,98.4408 255.185,96.2219 252.906,94.2203 
 250.439,92.4563 247.809,90.9473 245.041,89.7085 242.163,88.7522 239.204,88.0881 236.193,87.7226 233.161,87.6595 230.138,87.8995 227.154,88.4401 224.238,89.2758 
 221.421,90.3985 218.73,91.7968 216.192,93.4567 213.832,95.3618 211.675,97.4929 209.741,99.8287 208.049,102.346

In [12]:
model = Dense(2, 3, σ)

Dense(2, 3, NNlib.σ)

In [13]:
model.W

Tracked 3×2 Array{Float64,2}:
  0.870099   0.791934
 -0.333706  -1.05544 
  0.530088  -0.189723

Each of the 6 lines in the figure denotes a weight of the neuron on the right, taking as input the output of the neuron on the left. These weights are collected in the **matrix** `W`. Note that it seems to be "backwards", since it is designed to multiply vectors of length 2 (the input size):

In [16]:
x = rand(2)
model.W * x

Tracked 3-element Array{Float64,1}:
  1.06859 
 -0.779776
  0.320209

The whole `model` object represents a set of three sigmoidal neurons:

In [17]:
model(x)

Tracked 3-element Array{Float64,1}:
 0.744329
 0.314368
 0.579375

In [19]:
σ.(model.W*x + model.b)

Tracked 3-element Array{Float64,1}:
 0.744329
 0.314368
 0.579375

Note that here we have used Julia's **broadcasting** capability, in which the function $\sigma$ is applied to each element of the vector `W * x` in turn. This elementwise application of the function is implicit in most of the literature on machine learning, but it is much clearer to make it explicit, as Julia allows us (in fact, basically forces us) to do.

In [20]:
loss(x, y) = Flux.mse(model(x), y)

loss (generic function with 1 method)

In [36]:
data = zip(xs, ys)

Base.Iterators.Zip2{Array{Array{Float64,1},1},Array{Flux.OneHotVector,1}}(Array{Float64,1}[[0.708703, 0.341998], [0.648376, 0.284163], [0.647237, 0.282579], [0.647963, 0.283689], [0.647653, 0.2846], [0.648491, 0.28597], [0.647974, 0.285646], [0.649307, 0.287323], [0.648141, 0.286103], [0.64984, 0.288396]  …  [0.722939, 0.414788], [0.721564, 0.413433], [0.723195, 0.413965], [0.722358, 0.413741], [0.723049, 0.416157], [0.722233, 0.414729], [0.722148, 0.41648], [0.721761, 0.416422], [0.722839, 0.417423], [0.722266, 0.417273]], Flux.OneHotVector[Bool[true, false, false], Bool[true, false, false], Bool[true, false, false], Bool[true, false, false], Bool[true, false, false], Bool[true, false, false], Bool[true, false, false], Bool[true, false, false], Bool[true, false, false], Bool[true, false, false]  …  Bool[false, false, true], Bool[false, false, true], Bool[false, false, true], Bool[false, false, true], Bool[false, false, true], Bool[false, false, true], Bool[false, false, true], Bool[fa

In [37]:
collect(data)

2935-element Array{Tuple{Array{Float64,1},Flux.OneHotVector},1}:
 ([0.708703, 0.341998], Bool[true, false, false])
 ([0.648376, 0.284163], Bool[true, false, false])
 ([0.647237, 0.282579], Bool[true, false, false])
 ([0.647963, 0.283689], Bool[true, false, false])
 ([0.647653, 0.2846], Bool[true, false, false])  
 ([0.648491, 0.28597], Bool[true, false, false]) 
 ([0.647974, 0.285646], Bool[true, false, false])
 ([0.649307, 0.287323], Bool[true, false, false])
 ([0.648141, 0.286103], Bool[true, false, false])
 ([0.64984, 0.288396], Bool[true, false, false]) 
 ([0.648446, 0.287733], Bool[true, false, false])
 ([0.709808, 0.322328], Bool[true, false, false])
 ([0.650164, 0.290677], Bool[true, false, false])
 ⋮                                               
 ([0.722031, 0.412562], Bool[false, false, true])
 ([0.730362, 0.422273], Bool[false, false, true])
 ([0.722939, 0.414788], Bool[false, false, true])
 ([0.721564, 0.413433], Bool[false, false, true])
 ([0.723195, 0.413965], Bool[false,

In [38]:
params(model)

2-element Array{Any,1}:
 param([0.870099 0.791934; -0.333706 -1.05544; 0.530088 -0.189723])
 param([0.0, 0.0, 0.0])                                            

In [39]:
opt = SGD(params(model), 0.01)
# give a list of the parameters that will be modified

(::#71) (generic function with 1 method)

In [54]:
for i in 1:100
    Flux.train!(loss, data, opt)
end

In [55]:
model.W

Tracked 3×2 Array{Float64,2}:
  7.39953   -5.39071
 -4.46358  -10.173  
 -2.78041   14.7777 

In [42]:
model.b

Tracked 3-element Array{Float64,1}:
 -2.55731
  2.7284 
 -2.25216

Let's visualize the hyperplanes that the neurons have learnt.

In [56]:
plot()

contour!(0:0.01:1, 0:0.01:1, (x,y)->model([x,y]).data[1], levels=[0.5, 0.501], c=:blue)
contour!(0:0.01:1, 0:0.01:1, (x,y)->model([x,y]).data[2], levels=[0.5,0.501], c=:red)
contour!(0:0.01:1, 0:0.01:1, (x,y)->model([x,y]).data[3], levels=[0.5,0.501], c=:green)

scatter!(first.(x_apples), last.(x_apples), m=:cross, label="apples")
scatter!(first.(x_bananas), last.(x_bananas), m=:circle, label="bananas")
scatter!(first.(x_grapes), last.(x_grapes), m=:square, label="grapes")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 1.00 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 1.00 
 
<polyline clip-path="url(#clip6802)" style="stroke:#ba3655; stroke-width:1; stroke-opacity:1; fill:none" points="
 274.64,365.565 275.201,364.983 277.947,362.131 279.74,360.269 281.255,358.696 284.279,355.555 284.561,355.262 287.868,351.827 288.818,350.84 291.175,348.393 
 293.357,346.126 294.482,344.958 297.789,341.524 297.897,341.411 301.095,338.089 302.436,336.697 304.402,334.655 306.975,331.983 307.709,331.22 311.016,327.786 
 311.514,327.268 314.323,324.351 316.053,322.554 317.63,320.917 320.592,317.84 320.937,317.482 324.243,314.048 325.132,313.125 327.55,310.613 329.671,308.411 
 330.858,307.179 334.164,303.744 334.21,303.697 337.471,300.31 338.749,298.982 340.778,296.875 343.288,294.268 344.085,293.441 347.392,290.006 347.827,289.554 
 350.698,286.572 352.366,284.839 354.006,283.137 356.906,280.125 357.312,279.703 360.619,276.268 361.445,275.41 363.926,272.833 365.984,270.696 367.233,269.399 
 370.523,265.982 370.54,265.964 373.846,262.53 375.062,261.267 377.153,259.095 379.601,256.553 380.461,255.661 383.767,252.226 384.14,251.839 387.074,248.792 
 388.68,247.124 390.381,245.357 393.219,242.41 393.688,241.923 396.994,238.488 397.758,237.696 400.301,235.054 402.297,232.981 403.609,231.619 406.836,228.267 
 406.915,228.185 410.222,224.75 411.375,223.553 413.529,221.316 415.915,218.838 416.836,217.881 420.143,214.447 420.454,214.124 423.449,211.012 424.993,209.41 
 426.757,207.578 429.532,204.695 430.063,204.143 433.37,200.709 434.071,199.981 436.677,197.274 438.61,195.267 439.984,193.84 443.149,190.552 443.291,190.405 
 446.597,186.971 447.689,185.838 449.904,183.536 452.228,181.123 453.212,180.102 456.518,176.667 456.767,176.409 459.825,173.233 461.306,171.695 463.132,169.798 
 465.845,166.98 466.439,166.364 469.745,162.929 470.384,162.266 473.052,159.495 474.923,157.552 476.36,156.06 479.463,152.837 479.666,152.626 482.973,149.191 
 484.002,148.123 486.28,145.757 488.541,143.409 489.587,142.322 492.894,138.888 493.08,138.694 496.2,135.453 497.619,133.98 499.508,132.019 502.158,129.266 
 502.814,128.584 506.121,125.15 506.698,124.551 
 "/>
<polyline clip-path="url(#clip6802)" style="stroke:#ba3655; stroke-width:1; stroke-opacity:1; fill:none" points="
 506.698,124.806 506.366,125.15 503.06,128.584 502.158,129.52 499.753,132.019 497.619,134.235 496.446,135.453 493.139,138.888 493.08,138.949 489.832,142.322 
 488.541,143.663 486.525,145.757 484.002,148.378 483.218,149.191 479.912,152.626 479.463,153.092 476.605,156.06 474.923,157.807 473.298,159.495 470.384,162.521 
 469.991,162.929 466.684,166.364 465.845,167.235 463.377,169.798 461.306,171.95 460.07,173.233 456.767,176.664 456.764,176.667 453.457,180.102 452.228,181.378 
 450.15,183.536 447.689,186.093 446.843,186.971 443.536,190.405 443.149,190.807 440.229,193.84 438.61,195.521 436.922,197.274 434.071,200.236 433.615,200.709 
 430.309,204.143 429.532,204.95 427.002,207.578 424.993,209.664 423.695,211.012 420.454,214.379 420.388,214.447 417.081,217.881 415.915,219.093 413.774,221.316 
 411.375,223.807 410.467,224.75 407.161,228.185 406.836,228.522 403.854,231.619 402.297,233.236 400.547,235.054 397.758,237.95 397.24,238.488 393.933,241.923 
 393.219,242.665 390.626,245.357 388.68,247.379 387.319,248.792 384.14,252.094 384.012,252.226 380.706,255.661 379.601,256.808 377.399,259.095 375.062,261.522 
 374.092,262.53 370.785,265.964 370.523,266.237 367.478,269.399 365.984,270.951 364.171,272.833 361.445,275.665 360.864,276.268 357.558,279.703 356.906,280.38 
 354.251,283.137 352.366,285.094 350.944,286.572 347.827,289.808 347.637,290.006 344.33,293.441 343.288,294.523 341.023,296.875 338.749,299.237 337.716,300.31 
 334.41,303.744 334.21,303.952 331.103,307.179 329.671,308.666 327.796,310.613 325.132,313.38 324.489,314.048 321.182,317.482 320.592,3

We see that the result is not very good: this network is so simple that it is *not capable of learning a good representation of the data*. The reason for this is that the class of functions modelled is not complex enough.

Note that two of the hyperplanes have been learnt correctly, the one that separates bananas from the rest, and the one that separates grapes from the rest. The third hyperplane has not, and it is intuitively clear why: there *is no way* to separate apples from non-apples with a *single* hyperplane, given this data.